In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from pathlib import Path

import flexiznam as flz
from cottage_analysis.analysis import spheres
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, closed_loop_rsof, get_session_list
from v1_depth_analysis.v1_manuscript_2023 import common_utils as v1_common_utils

In [ ]:
VERSION = 3
SAVE_ROOT = (
    Path(
        f"/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
    )
    / "fig4"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# Concatenate all sessions
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
results_all = v1_common_utils.concatenate_all_neurons_df(
    flexilims_session=flexilims_session,
    session_list=get_session_list.get_sessions(
        flexilims_session,
        closedloop_only=False,
        openloop_only=True,
    ),
    filename="neurons_df_openclosed.pickle",
    cols=[
        "roi",
        "best_depth",
        "preferred_depth_closedloop",
        "preferred_depth_closedloop_crossval",
        "depth_tuning_test_rsq_closedloop",
        "depth_tuning_test_spearmanr_pval_closedloop",
        "depth_tuning_test_spearmanr_rval_closedloop",
        "preferred_RS_closedloop_g2d_openclosed0",
        "preferred_OF_closedloop_g2d_openclosed0",
        "rsof_rsq_closedloop_g2d_openclosed0",
        "preferred_RS_openloop_actual_g2d_openclosed0",
        "preferred_OF_openloop_actual_g2d_openclosed0",
        "rsof_rsq_openloop_actual_g2d_openclosed0",
        "rsof_popt_closedloop_g2d_openclosed0",
        "rsof_popt_openloop_actual_g2d_openclosed0",
    ],
    read_iscell=True,
    verbose=True,
)
# results_all["preferred_RS_closedloop_g2d"] = (
#     results_all["preferred_RS_closedloop_g2d"] * 100
# )
# results_all["preferred_depth_closedloop"] = (
#     results_all["preferred_depth_closedloop"] * 100
# )
# results_all["preferred_OF_closedloop_g2d"] = np.degrees(
#     results_all["preferred_OF_closedloop_g2d"]
# )
# results_all["preferred_RS_openloop_actual_g2d"] = (
#     results_all["preferred_RS_openloop_actual_g2d"] * 100
# )
# results_all["preferred_OF_openloop_actual_g2d"] = np.degrees(
#     results_all["preferred_OF_openloop_actual_g2d"]
# )

In [ ]:
closedloop_popt = np.stack(results_all.rsof_popt_closedloop_g2d_openclosed0)
closedloop_amp = np.exp(closedloop_popt[:, 0])
openloop_popt = np.stack(results_all.rsof_popt_openloop_actual_g2d_openclosed0)
openloop_amp = np.exp(openloop_popt[:, 0])

In [ ]:
select_neurons = (results_all.iscell==1) & (results_all.depth_tuning_test_spearmanr_pval_closedloop<0.05) & (results_all.depth_tuning_test_spearmanr_rval_closedloop>0)
plt.subplot(121)
plt.scatter(closedloop_amp[select_neurons], openloop_amp[select_neurons], s=1, alpha=0.5, c='k')
plt.plot(np.arange(100), np.arange(100), 'r--')
plt.xscale('log')
plt.yscale('log')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('Closed-loop amplitude')
plt.ylabel('Open-loop amplitude')

plt.subplot(122)
select_neurons = (results_all.iscell==1) & (results_all.depth_tuning_test_spearmanr_pval_closedloop<0.05) & (results_all.depth_tuning_test_spearmanr_rval_closedloop>0.2)
plt.scatter(closedloop_amp[select_neurons], openloop_amp[select_neurons], s=1, alpha=0.5, c='k')
plt.plot(np.arange(100), np.arange(100), 'r--')
plt.xscale('log')
plt.yscale('log')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('Closed-loop amplitude')
plt.ylabel('Open-loop amplitude')
plt.tight_layout()

In [ ]:
select_range = ((select_neurons)
                & (closedloop_amp>0.1) & (closedloop_amp<50)
                & (openloop_amp>0.1) & (openloop_amp<50))
plt.hist(closedloop_amp[select_range]-openloop_amp[select_range], bins=100, color='k', alpha=0.5)
print(np.median(closedloop_amp[select_range]-openloop_amp[select_range]))
